In [2]:
# Import Packages

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(website_url,'lxml')

In [92]:
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [107]:
print(soup.find('div',class_=''))

<div id="contentSub"></div>


wikitable sortable

In [6]:
#Create My_table from wiki website

My_table = soup.find("table",{"class":"wikitable sortable"})
print(type(My_table))

<class 'bs4.element.Tag'>


In [7]:
# Create divs from wiki website 

divs = soup.findAll("table", {'class':'wikitable sortable'})
print(type(divs))

<class 'bs4.element.ResultSet'>


In [22]:
#Investigate divs

In [31]:
# Find Rows

for div in divs:
    rows = div.findAll('tr')

In [35]:
print(type(rows))

print(type(rows[0]))

print(rows[0].text)

print(rows[1].text)

#rows[0] 

<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>

Postal Code

Borough

Neighborhood


M1A

Not assigned





In [40]:
# Find Column Names
# first row start with th

for row in rows[0:1]:
    
    elements = row.find_all('th')
    
    elements = [element.text.strip() for element in elements]
    
    print(elements)

['Postal Code', 'Borough', 'Neighborhood']


In [37]:
print(type(elements))

<class 'list'>


In [49]:
# Create dummy DataFrame with 3 columns

d = [[0,0,0]]

df = pd.DataFrame(data=d, columns=list('ABC'))

df

,A,B,C
0,0,0,0


In [50]:
# Find Elements in the table
# except first row start with td

for row in rows[1:]:
    
    elements = row.find_all('td')
    
    elements = [element.text.strip() for element in elements]

    row_df = pd.DataFrame([elements],columns=list('ABC'))
    
    # concat rows to df dataframe
    
    df = df.append(row_df, ignore_index=True)
    
df.head()

,A,B,C
0,0,0,0
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [51]:
# Drop first row

df = df.drop([0])
df.head()

,A,B,C
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [52]:
# change column names

df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [53]:
df.shape

(180, 3)

In [54]:
# Drop cells with a borough that is Not assigned.

df1 = df[df.Borough != "Not assigned"]
df1.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [56]:
# Check nan values

df1[df1.Neighborhood == np.nan]

,PostalCode,Borough,Neighborhood


In [57]:
df1.shape

(103, 3)

# Problem 2

In [60]:
ls

Geospatial_Coordinates.csv  Week3.ipynb


In [73]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv") 
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [85]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [86]:
merged_df = pd.merge(df1, geo_df, on='PostalCode')

In [87]:
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Problem 3

In [132]:
# pip install geopy
# pip install folium

In [119]:
Toronto_df = merged_df[merged_df['Borough'].str.contains("Toronto")]
Toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [127]:
# set number of clusters


toronto_grouped_clustering = Toronto_df.drop(['Neighborhood','Borough','PostalCode'], 1)
toronto_grouped_clustering.head()

,Latitude,Longitude
2,43.654260,-79.360636
4,43.662301,-79.389494
9,43.657162,-79.378937
15,43.651494,-79.375418
19,43.676357,-79.293031


In [128]:
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 3, 1, 1, 2, 1, 2], dtype=int32)

In [133]:
# Add cluster labels

toronto_grouped_clustering['Cluster Labels'] = kmeans.labels_
toronto_grouped_clustering.head()

,Latitude,Longitude,Cluster Labels
2,43.654260,-79.360636,1
4,43.662301,-79.389494,1
9,43.657162,-79.378937,1
15,43.651494,-79.375418,1
19,43.676357,-79.293031,3


In [114]:
import folium

In [137]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(toronto_grouped_clustering['Latitude'], toronto_grouped_clustering['Longitude'], toronto_grouped_clustering['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters